In [ ]:
# Create Sagemaker execution role
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os
from sklearn.metrics import accuracy_score
import numpy as np
import pickle

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

In [ ]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.cnn")).upload_file(
    train_dir + "/train.cnn"
)

In [ ]:
# This is not required as these values are the defaults:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Modify this based on your script name !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
entry_point = "cnnScript.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.xlarge"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "script",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "cnn-model",
}

estimator = SKLearn(**estimator_parameters)

In [ ]:
inputs = {
    "train": pickle_train_s3_uri
}

# starting the training job
estimator.fit(inputs)

In [ ]:
sklearn_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.xlarge',
                                     endpoint_name='cnn-endpoint')

In [ ]:
def getAccuracyOfPrediction(cnn_predictions, test_labels):
    cnn_predicted_labels = np.argmax(cnn_predictions, axis=1)
    accuracy = accuracy_score(test_labels, cnn_predicted_labels)
    print("Accuracy:", accuracy)

In [ ]:
def getTestData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    fileNameTestBatch = './cifar-10-batches-py/test_batch'
    test_data = unpickle(fileNameTestBatch)[b'data']
    test_data = test_data.reshape(-1, 32, 32, 3) / 255.0
    test_labels = np.array(unpickle(fileNameTestBatch)[b'labels'])

    num_samples_to_select = 600
    random_indices = np.random.choice(test_data.shape[0], num_samples_to_select, replace=False)
    selected_test_data = test_data[random_indices]
    selected_test_labels = test_labels[random_indices]

    return selected_test_data, selected_test_labels

In [ ]:
test_data, test_labels = getTestData()

In [ ]:
predictions = sklearn_predictor.predict(test_data)

In [ ]:
getAccuracyOfPrediction(predictions, test_labels)

In [ ]:
sklearn_predictor.delete_endpoint(True)